In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import datetime as dt
import matplotlib.dates as mdates
df = pd.read_csv('2016-donations.csv')
df
df.head()
df = pd.read_csv('2016-donations.csv', index_col=False)
df.shape
df.describe()
df.columns
df['contb_receipt_amt']
df.contb_receipt_amt
df.contb_receipt_amt.head()
contributions = df[['cand_nm', 'contb_receipt_dt', 'contb_receipt_amt', 'contbr_nm', 'election_tp']]
contributions
sum(pd.isnull(contributions['election_tp']))
clean_contributions = contributions.dropna()
clean_contributions.shape
df.shape[0] - clean_contributions.shape[0]
clean_contributions.head()
clean_contributions.cand_nm.unique()
trump_conts = clean_contributions[clean_contributions.cand_nm == 'Trump, Donald J.']
trump_conts.head()
trump_conts.describe()
bernie_conts = clean_contributions[clean_contributions.cand_nm == 'Sanders, Bernard']
bernie_conts.describe()
hillary_conts = clean_contributions[clean_contributions.cand_nm == 'Clinton, Hillary Rodham']
hillary_conts.describe()
hillary_conts_short = hillary_conts[abs(hillary_conts.contb_receipt_amt) > 100]
trump_conts_short = trump_conts[abs(trump_conts.contb_receipt_amt) > 100]
def to_donation_xy_points(contributions):
    donations_dict = defaultdict(lambda:0)
    for index, row in contributions.iterrows():
        date = dt.datetime.strptime(row.contb_receipt_dt, '%d-%b-%y')
        donations_dict[date] += row.contb_receipt_amt
    sorted_by_date = sorted(donations_dict.items())
    return zip(*sorted_by_date)
hill_x, hill_y = to_donation_xy_points(hillary_conts_short)
plt.figure(figsize=(20,10))
plt.plot(hill_x, hill_y)
plt.show()
hillary_conts_short.sort_values('contb_receipt_amt', ascending=False)
hillary_conts_real = hillary_conts_short[hillary_conts_short.contbr_nm != 'HILLARY VICTORY FUND - UNITEMIZED']
hill_x2, hill_y2 = to_donation_xy_points(hillary_conts_real)
plt.figure(figsize=(20,10))
plt.plot(hill_x2, hill_y2)
plt.show()
plt.axvline(x=dt.datetime(2016,7,26), linewidth=0.3) #dnc opens with bernie endorsement
plt.axvline(x=dt.datetime(2016,9,26), linewidth=0.3) #presidential debate
plt.axvline(x=dt.datetime(2016,9,30), linewidth=0.3) #polls show hillary is ahead after first debate
trump_x, trump_y = to_donation_xy_points(trump_conts_short)
plt.figure(figsize=(20,10))
plt.plot(hill_x2, hill_y2, label='hillary', c='blue')
plt.plot(trump_x, trump_y, label='trump', c='red')
plt.axvline(x=dt.datetime(2016,6,21), c='pink') # trump speech at rnc
plt.axvline(x=dt.datetime(2016,10,20), c='pink') # presidential debate
plt.legend()
plt.show()
tweets = pd.read_csv('trump-tweets.csv')
tweets.head()
android = tweets[tweets.source.str.contains('Android')]
iphone = tweets[tweets.source.str.contains('iPhone')]
sum(iphone.text.str.contains('http')) / len(iphone)
plt.bar(0, sum(iphone.text.str.contains('http')) / len(iphone), width=0.5, label='iphone')
plt.bar(1, sum(android.text.str.contains('http')) / len(android), width=0.5, label='android')
plt.legend()
plt.show()
search_string = '(?i)hillary'
plt.bar(0, sum(iphone.text.str.contains(search_string)) / len(iphone) * 100, width=0.5, label='iphone')
plt.bar(1, sum(android.text.str.contains(search_string)) / len(android) * 100, width=0.5, label='android')
plt.legend()
plt.show()
search_string = '(?i)crooked'
search_string = '(?i)sad'
search_string = '(?i)lies'
search_string = '(?i)jobs'
search_string = '(?i)business'
tweets.created_at = tweets.created_at.astype(str).apply(lambda x: x[:10])
def for_trump_tweets(dataframe, field):
    tweets_dict = defaultdict(lambda:0)
    for index, row in dataframe.iterrows():
        date = dt.datetime.strptime(row.created_at, '%m-%d-%Y')
        tweets_dict[date] += row[field]
    sorted_by_date = sorted(tweets_dict.items())
    return zip(*sorted_by_date)
tweet_x, tweet_y = for_trump_tweets(tweets, 'favorite_count')
tweet_x2, tweet_y2 = for_trump_tweets(tweets, 'retweet_count')
plt.figure(figsize=(20,10))
plt.scatter(tweet_x, tweet_y, s=np.asarray(tweet_y) / 5000, label='favorites', c='red')
plt.scatter(tweet_x2, np.asarray(tweet_y2) * 2, s=np.asarray(tweet_y2) / 2000, label='retweets', c='green')
plt.plot(trump_x, trump_y, label='donations')
plt.legend()
plt.show()
plt.xlim(left=dt.datetime(2016,5,1))